In [26]:
from feast import FeatureStore

In [ ]:
fs = FeatureStore(repo_path="../client_feature_repo")

In [28]:
from datetime import datetime

fs.materialize_incremental(end_date=datetime.now())

Materializing 15 feature views to 2025-12-04 09:00:40+00:00 into the remote online store.

transaction_7d_aggregations from 2025-11-27 09:00:44+00:00 to 2025-12-04 09:00:40+00:00:
branch_performance from 2025-11-04 09:00:53+00:00 to 2025-12-04 09:00:40+00:00:
atm_usage_30d from 2025-11-04 09:01:01+00:00 to 2025-12-04 09:00:40+00:00:
atm_location_performance from 2025-11-04 09:01:11+00:00 to 2025-12-04 09:00:40+00:00:
branch_service_preferences from 2025-06-07 09:01:20+00:00 to 2025-12-04 09:00:40+00:00:


/opt/app-root/lib64/python3.12/site-packages/feast/utils.py:249: UserWarning: Discarding nonzero nanoseconds in conversion.
  return ts.to_pydatetime()


call_center_90d from 2025-09-05 09:01:32+00:00 to 2025-12-04 09:00:40+00:00:


/opt/app-root/lib64/python3.12/site-packages/feast/utils.py:249: UserWarning: Discarding nonzero nanoseconds in conversion.
  return ts.to_pydatetime()


transaction_30d_aggregations from 2025-11-04 09:01:44+00:00 to 2025-12-04 09:00:40+00:00:
customer_transaction_interaction from 2025-11-04 09:01:53+00:00 to 2025-12-04 09:00:40+00:00:
atm_time_patterns from 2025-10-05 09:02:06+00:00 to 2025-12-04 09:00:40+00:00:
transaction_90d_patterns from 2025-09-05 09:02:15+00:00 to 2025-12-04 09:00:40+00:00:


...

call_center_predictive from 2025-06-07 09:02:55+00:00 to 2025-12-04 09:00:40+00:00:


/opt/app-root/lib64/python3.12/site-packages/feast/utils.py:249: UserWarning: Discarding nonzero nanoseconds in conversion.
  return ts.to_pydatetime()


branch_visits_90d from 2025-09-05 09:03:07+00:00 to 2025-12-04 09:00:40+00:00:


/opt/app-root/lib64/python3.12/site-packages/feast/utils.py:249: UserWarning: Discarding nonzero nanoseconds in conversion.
  return ts.to_pydatetime()


transaction_details from 2025-11-27 09:03:18+00:00 to 2025-12-04 09:00:40+00:00:


In [29]:
import pandas as pd
from datetime import datetime

customer_id = "CUST_000001"

entity_df = pd.DataFrame({
    "customer_id": [customer_id],
    "event_timestamp": [datetime.now()]
})

try:
    historical = fs.get_historical_features(
        entity_df=entity_df,
        features=[
            "call_center_predictive:call_type",
            "call_center_predictive:call_duration_minutes",
            "call_center_predictive:is_resolved",
            "call_center_predictive:customer_satisfaction_score"
        ]
    )
    print("✓ SUCCESS - Historical features retrieved")
    print(historical.to_df())
except Exception as e:
    print(f"✗ FAILED: {e}")

✓ SUCCESS - Historical features retrieved
   customer_id                  event_timestamp          call_type  \
0  CUST_000001 2025-12-04 09:08:14.848749+00:00  TECHNICAL_SUPPORT   

   call_duration_minutes  is_resolved  customer_satisfaction_score  
0                    2.9            1                          2.4  


In [30]:
print("\nTesting get_online_features with branch_visits_90d...")
try:
    online_branch = fs.get_online_features(
        entity_rows=[{"customer_id": customer_id}],
        features=[
            "branch_visits_90d:service_type",
            "branch_visits_90d:wait_time_minutes",
            "branch_visits_90d:customer_satisfaction_score"
        ]
    )
    print("✓ SUCCESS - Online features retrieved")
    print(online_branch.to_df())
except Exception as e:
    print(f"✗ FAILED: {e}")


Testing get_online_features with branch_visits_90d...
✓ SUCCESS - Online features retrieved
   customer_id  wait_time_minutes     service_type  \
0  CUST_000001               43.9  ACCOUNT_OPENING   

   customer_satisfaction_score  
0                          1.2  


In [31]:
from feast import FeatureStore, FeatureView, Field, FileSource, Entity
from feast.types import Int64, String
from feast.value_type import ValueType
from datetime import timedelta
import pandas as pd
from datetime import datetime


print("\n[1] Creating FeatureView definition...")

# Get existing customer entity from feature store
try:
    customer_entity = fs.get_entity("customer")
    print(f"✓ Found existing customer entity: {customer_entity.name}")
except Exception as e:
    print(f"✗ Could not get customer entity: {e}")
    print("  Creating customer entity...")
    # Create customer entity if it doesn't exist
    customer_entity = Entity(
        name="customer",
        join_keys=["customer_id"],
        value_type=ValueType.STRING,
        description="Customer entity for testing"
    )
    fs.apply([customer_entity])
    print("✓ Created and applied customer entity")

# Get or create data source (using existing customer data)
try:
    customer_data_source = fs.get_data_source("customer_data_source")
    print(f"✓ Found existing data source: {customer_data_source.name}")
except Exception as e:
    print(f"  Creating data source...")
    customer_data_source = FileSource(
        name="customer_data_source",
        path="data/customers.parquet",
        timestamp_field="created_timestamp"
    )
    fs.apply([customer_data_source])
    print(f"✓ Created and applied data source: {customer_data_source.name}")

# Create a simple test FeatureView
test_feature_view = FeatureView(
    name="test_customer_simple_fv",  # Unique name for testing
    entities=[customer_entity],
    ttl=timedelta(days=30),
    schema=[
        Field(
            name="age",
            dtype=Int64,
            description="Customer age"
        ),
        Field(
            name="credit_score",
            dtype=Int64,
            description="Customer credit score"
        ),
    ],
    source=customer_data_source,
    description="Simple test feature view created programmatically"
)

print(f"✓ FeatureView '{test_feature_view.name}' defined")
#print(f"  - Entities: {[e.name for e in test_feature_view.entities]}")
print(f"  - Features: {[f.name for f in test_feature_view.schema]}")

# ============================================================================
# Step 2: Apply FeatureView to the feature store
# ============================================================================
print("\n[2] Applying FeatureView to feature store...")
try:
    fs.apply([test_feature_view])
    print("✓ FeatureView applied successfully")
except Exception as e:
    print(f"✗ Failed to apply FeatureView: {e}")
    import traceback
    traceback.print_exc()
    exit(1)

# ============================================================================
# Step 3: Verify FeatureView was created
# ============================================================================
print("\n[3] Verifying FeatureView exists...")
fs.refresh_registry()
try:
    fv = fs.get_feature_view("test_customer_simple_fv")
    print(f"✓ FeatureView '{fv.name}' found in registry")
except Exception as e:
    print(f"✗ FeatureView not found: {e}")

# ============================================================================
# Step 4: Test Historical Features Retrieval
# ============================================================================
print("\n[4] Testing get_historical_features...")
customer_id = "CUST_000001"
entity_df = pd.DataFrame({
    "customer_id": [customer_id],
    "event_timestamp": [datetime.now()]
})

try:
    historical = fs.get_historical_features(
        entity_df=entity_df,
        features=[
            "test_customer_simple_fv:age",
            "test_customer_simple_fv:credit_score"
        ]
    )
    print("✓ SUCCESS - Historical features retrieved")
    print(historical.to_df())
except Exception as e:
    print(f"✗ FAILED: {e}")
    import traceback
    traceback.print_exc()


[1] Creating FeatureView definition...
✓ Found existing customer entity: customer
✓ Found existing data source: customer_data_source
✓ FeatureView 'test_customer_simple_fv' defined
  - Features: ['credit_score', 'age']

[2] Applying FeatureView to feature store...
✓ FeatureView applied successfully

[3] Verifying FeatureView exists...
✓ FeatureView 'test_customer_simple_fv' found in registry

[4] Testing get_historical_features...
✓ SUCCESS - Historical features retrieved
Empty DataFrame
Columns: [customer_id, event_timestamp, age, credit_score]
Index: []
